In [1]:
from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw

In [209]:
from netgen.occ import *

box = Rectangle(6,2).Face()
circ1 = Circle((1.5,0.5), 0.3).Face()
circ2 = Circle((4.5,1.5), 0.3).Face()


spline2 = SplineApproximation([(1.3,0.3) , (2.5, 0.8) , (3.5,1.2), (4.5,1.3), (4.2,1.6),
                              ])

wp = WorkPlane().MoveTo(1.8,0.5)
wp.Line(0.2).Arc(.3, 15)
wp.Line(2).Arc(0.3, 180)
wp.Line(2).Arc(0.3, 160)
wp.Close()
sp1 = wp.Face()


In [257]:
wp = WorkPlane()
wp.RectangleC(6,2).RectangleC(0.5,0.1).Reverse().Close()
sp1 = wp.Face()

In [258]:


# air = box-circ1-circ2 -sp1
air = box - sp1
circ1.faces.name = "left"
circ2.faces.name = "right"
circ2.edges.name = "cr"
circ1.edges.name = "cl"
air.faces.name = "air"
air.edges.Min(Y).name ='bottom'
air.edges.Max(Y).name ='top'
air.edges.Min(X).name ='left'
air.edges.Max(X).name ='right'

shape = Glue([air])
# Draw (shape);
air = OCCGeometry(shape, dim=2)

mesh = Mesh(air.GenerateMesh(maxh=0.1)).Curve(3)
Draw (mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

In [140]:
mesh.GetBoundaries()

('bottom',
 'left',
 'right',
 'top',
 'default',
 'default',
 'default',
 'default',
 'cr',
 'cr',
 'default',
 'default',
 'default',
 'cl',
 'cl',
 'default',
 'default')

In [141]:

# H1-conforming finite element space
fes = H1(mesh, order=3, dirichlet="top", autoupdate=True)
# define trial- and test-functions
u = fes.TrialFunction()
v = fes.TestFunction()

In [142]:
# the bilinear-form 
a = BilinearForm(grad(u)*grad(v)*dx)

funcf = 50*x*y
funcf = 50*sin(y)
f = LinearForm(funcf*v*dx)

a.Assemble()
f.Assemble()
Draw(funcf,mesh)


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

In [143]:
gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs()) * f.vec
Draw(gfu);
# Draw(grad(gfu),mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…